# MANUAL TEST TO ADD TO THE API

## STEP: 2

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("src/data/raw/bank_enriched_addresses.csv")

In [ ]:
df.head()

In [20]:
#RULE 2.a
rule2a_col = "pdays"
if rule2a_col in df.columns:
    df = df[df[rule2a_col] != -1]
else:
    print(f"No {rule2a_col} column.")
    
print(df[rule2a_col])

1       339
2       330
5       176
6       330
9       147
       ... 
4492    234
4504    370
4514    153
4519    211
4520    249
Name: pdays, Length: 816, dtype: int64


In [21]:
#RULE 2.b
rule2b_col = "name"
if rule2b_col in df.columns:
    df[['first_name', 'second_name']] = df[rule2b_col].str.split(' ', 1, expand=True)
    
    df.drop(rule2b_col, axis=1, inplace=True)
else:
    print(f"No {rule2b_col} column.")
    
print(df[['first_name', 'second_name']])

     first_name second_name
1      domenico       swift
2         talon   gulgowski
5      richmond     effertz
6        lauren     mcclure
9       justina      willms
...         ...         ...
4492       erna   o'connell
4504    johnson        beer
4514     maudie  wintheiser
4519  zachariah       sipes
4520      julio      deckow

[816 rows x 2 columns]


/tmp/ipykernel_484252/2576133268.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['first_name', 'second_name']] = df[rule2b_col].str.split(' ', 1, expand=True)


In [22]:
#RULE 2.c
from src.app.utils.utils import Utils

rule2c_col = "age"
if rule2c_col in df.columns:
    df[rule2c_col] = [ Utils.age_to_bucket(age) for age in df[rule2c_col] ]
else:
    print(f"No {rule2c_col} column.")


In [55]:
df[rule2c_col].value_counts().to_json()

'{"30-39":343,"40-49":195,"50-59":142,"20-29":85,"60-69":25,"70-79":20,"80-89":6}'

In [23]:
#RULE 2.d
columns_2d = []
        
for col in df.columns:
    if df[col].iloc[0]=="yes" or df[col].iloc[0] == "no":
        columns_2d.append(col)

for col in columns_2d:
    df[col] = df[col].replace({'yes': True, 'no': False})

print(df[columns_2d])

      default  housing   loan      y
1       False     True   True  False
2       False     True  False  False
5       False    False  False  False
6       False     True  False  False
9       False     True   True  False
...       ...      ...    ...    ...
4492    False     True   True  False
4504    False     True   True   True
4514    False     True  False  False
4519    False    False  False  False
4520    False     True   True  False

[816 rows x 4 columns]


In [24]:
#RULE 2.e
month_map = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05',
        'jun': '06', 'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10',
        'nov': '11', 'dec': '12'}

df['month'] = df['month'].apply(lambda x: month_map.get(x.lower()))
df['date'] = df['day'].astype(str) + '/' + df['month']
print(df['date'])

1       11/05
2       16/04
5       23/02
6       14/05
9       17/04
        ...  
4492    16/04
4504    13/05
4514    20/04
4519     6/02
4520     3/04
Name: date, Length: 816, dtype: object


In [ ]:
df['date'].value_counts()

In [25]:
#RULE 2.f
df.rename(columns={'y': 'outcome'}, inplace=True)


## STEP: 3 

In [ ]:
df[df['address'].str.lower().str.contains("lake")]


In [26]:
#RULE 3.a/b/c

def categorize_address(address:str, category:str):
    water_keywords = ['lake', 'creek', 'river', 'spring', 'ocean', 'seashore', 'beach', 'coastal', 'waterfront']
    relief_keywords = ['hill', 'mountain', 'canyon', 'valley', 'cliff']
    flat_keywords = ['plain', 'plateau', 'field']

    address_lower = address.lower()
    
    if category == "water":
        return any(keyword in address_lower for keyword in water_keywords)
    elif category == 'relief':
        return any(keyword in address_lower for keyword in relief_keywords)
    elif category == 'flat':
        return any(keyword in address_lower for keyword in flat_keywords)
    

rule3_col = "address"
if rule3_col in df.columns:
    df["address_category_water"]  = [ categorize_address(address, "water")  for address in df[rule3_col] ]
    df["address_category_relief"] = [ categorize_address(address, "relief") for address in df[rule3_col] ]
    df["address_category_flat"]   = [ categorize_address(address, "flat")   for address in df[rule3_col] ]
else:
    print(f"No {rule3_col} column.")

df[["address_category_water","address_category_relief","address_category_flat"]]

,address_category_water,address_category_relief,address_category_flat
1,False,False,False
2,False,False,False
5,False,False,False
6,False,False,False
9,False,False,False
...,...,...,...
4492,False,False,False
4504,False,False,False
4514,False,False,False
4519,False,False,False


In [27]:
result = (df[df['address_category_water'].notna()]  # Filter non-empty feature values
          .groupby('address_category_water')  # Group by the 'feature' column
          .apply(lambda x: x.sort_values('age').count()))  # Apply sort and count within each group

In [28]:
result

,age,address,job,marital,education,default,balance,housing,loan,contact,...,pdays,previous,poutcome,outcome,first_name,second_name,date,address_category_water,address_category_relief,address_category_flat
address_category_water,,,,,,,,,,,,,,,,,,,,,
False,767,767,767,767,767,767,767,767,767,767,...,767,767,767,767,767,767,767,767,767,767
True,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49


In [29]:
result2 = (df[df['address_category_water'].notna()]  # Filter non-empty feature values
          .groupby('age')  # Group by the 'feature' column
          .apply(lambda x: x.sort_values('address_category_water').count()))  # Apply sort and count within each group
print(result2["address_category_water"])

age
20-29     85
30-39    343
40-49    195
50-59    142
60-69     25
70-79     20
80-89      6
Name: address_category_water, dtype: int64


In [49]:
import json
json.loads( result2["address_category_water"].to_json())

{'20-29': 85,
 '30-39': 343,
 '40-49': 195,
 '50-59': 142,
 '60-69': 25,
 '70-79': 20,
 '80-89': 6}

In [53]:
result2 = (df  # Filter non-empty feature values
          .groupby('age')  # Group by the 'feature' column
          )  # Apply sort and count within each group
print(result2["address_category_relief"])

age
20-29     85
30-39    343
40-49    195
50-59    142
60-69     25
70-79     20
80-89      6
Name: address_category_relief, dtype: int64


In [52]:
result2 = (df[df['address_category_relief'].notna()]  # Filter non-empty feature values
          .groupby('age')  # Group by the 'feature' column
          .apply(lambda x: x.sort_values('address_category_relief').count()))  # Apply sort and count within each group
print(result2["address_category_relief"])

age
20-29     85
30-39    343
40-49    195
50-59    142
60-69     25
70-79     20
80-89      6
Name: address_category_relief, dtype: int64


In [50]:
import json
json.loads( result2["address_category_relief"].to_json())

{'20-29': 85,
 '30-39': 343,
 '40-49': 195,
 '50-59': 142,
 '60-69': 25,
 '70-79': 20,
 '80-89': 6}

In [51]:
import json
json.loads( result2["address_category_flat"].to_json())

{'20-29': 85,
 '30-39': 343,
 '40-49': 195,
 '50-59': 142,
 '60-69': 25,
 '70-79': 20,
 '80-89': 6}

In [ ]:
for a, b in df[['first_name', 'second_name']].values:
    print(a, b)

In [5]:
from src.app.general.service import MainService

In [4]:
df_new = MainService.transformation(df)

NameError: name 'MainService' is not defined

In [ ]:
df["name"] = [ f"{a} {b}" for a, b in df[['first_name', 'second_name']].values]

In [ ]:
df.loc[:, ['first_name', 'second_name']] = df["name"].str.split(' ', 1, expand=True)

In [174]:
df[df["pdays"] != -1]

,age,name,address,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
1,33,domenico swift,"3887RATKEMEADOWS\NNORTHLEANNATON,VI94092-4948",services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,talon gulgowski,"056 pfannerstill cape st.\neast marta, nj 21058",management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
5,35,richmond effertz,"3547 Treutel Crossing Apt. 567\nReillytown, AS...",management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no
6,36,lauren mcclure,"177CorneliusCampApt.478\nNikolausbury,AA16457-...",self-employed,married,tertiary,no,307,yes,no,cellular,14,may,341,1,330,2,other,no
9,43,justina willms,"03353CORKERYPRAIRIE\NWESTLONNIE,AR87918-1164",services,married,primary,no,-88,yes,yes,cellular,17,apr,313,1,147,2,failure,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4492,32,erna o'connell,"7491KILBACKMEADOWAPT.792\NPAGACMOUTH,AS11081-6594",technician,single,secondary,no,309,yes,yes,cellular,16,apr,346,1,234,3,failure,no
4504,42,johnson beer,"7081 leffler wall ave.\nkemmerside, dc 00182",blue-collar,single,secondary,no,1080,yes,yes,cellular,13,may,951,3,370,4,failure,yes
4514,38,maudie wintheiser,"25096 William Fi3ld\nMcCull0ughm0uth, 0R 33380...",blue-collar,married,secondary,no,1205,yes,no,cellular,20,apr,45,4,153,1,failure,no
4519,28,zachariah sipes,"9234 BORER MEADOW AVE.\NPORT LUNABOROUGH, NJ 3...",blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [187]:
df_step2.columns

Index(['age', 'address', 'job', 'marital', 'education', 'default', 'balance',
       'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'outcome', 'first_name', 'second_name',
       'date'],
      dtype='object')

In [6]:
df_step2 = MainService.transformation_step2(df)
df_step3 = MainService.transformation_step3(df_step2)

step2
      age              name  \
0      30  giovanni kessler   
1      33    domenico swift   
2      35   talon gulgowski   
3      30       cayla pagac   
4      59         sim ratke   
...   ...               ...   
4516   33     kenton miller   
4517   57      ahmed corwin   
4518   57    antwon schaden   
4519   28   zachariah sipes   
4520   44      julio deckow   

                                                address            job  \
0             17864 Grace Field\nLonzoborough, IN 28057     unemployed   
1         3887RATKEMEADOWS\NNORTHLEANNATON,VI94092-4948       services   
2       056 pfannerstill cape st.\neast marta, nj 21058     management   
3        146 gay fort suite 622\nwest kathryn, la 81781     management   
4     779 Hegmann Harbors St.\nIsaacmouth, MS 13269-...    blue-collar   
...                                                 ...            ...   
4516  359 Weber Valley Apt. 488\nZiemannburgh, FL 01748       services   
4517         0005CAMILAMANORA

/home/leonardo/Documents/Dev/EarnestAnalytics/src/app/general/service.py:92: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['first_name', 'second_name']] = df[rule2b_col].str.split(' ', 1, expand=True)
/home/leonardo/Documents/Dev/EarnestAnalytics/src/app/general/service.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['first_name', 'second_name']] = df[rule2b_col].str.split(' ', 1, expand=True)
/home/leonardo/Documents/Dev/EarnestAnalytics/src/app/general/service.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [168]:
df_step3.to_parquet("src/data/export/test3.parquet")

In [11]:
df_new = pd.read_parquet("src/data/export/test2.parquet")

In [16]:
s = "a.b.c"
s.split(".")[-1]

'c'

In [15]:
a = [1, 2, 3]
print(a[-1])

3


In [13]:
print(df_new.columns)

Index(['age', 'address', 'job', 'marital', 'education', 'default', 'balance',
       'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'outcome', 'first_name', 'second_name',
       'date', 'address_category_water', 'address_category_relief',
       'address_category_flat'],
      dtype='object')


In [8]:
df_step2.to_parquet("src/data/export/test2.parquet")

In [9]:
df_new = pd.read_parquet("src/data/export/test2.parquet")

In [10]:
print(df_new)

         age                                            address  \
index                                                             
1      30-39      3887RATKEMEADOWS\NNORTHLEANNATON,VI94092-4948   
2      30-39    056 pfannerstill cape st.\neast marta, nj 21058   
5      30-39  3547 Treutel Crossing Apt. 567\nReillytown, AS...   
6      30-39  177CorneliusCampApt.478\nNikolausbury,AA16457-...   
9      40-49       03353CORKERYPRAIRIE\NWESTLONNIE,AR87918-1164   
...      ...                                                ...   
4492   30-39  7491KILBACKMEADOWAPT.792\NPAGACMOUTH,AS11081-6594   
4504   40-49       7081 leffler wall ave.\nkemmerside, dc 00182   
4514   30-39  25096 William Fi3ld\nMcCull0ughm0uth, 0R 33380...   
4519   20-29  9234 BORER MEADOW AVE.\NPORT LUNABOROUGH, NJ 3...   
4520   40-49       2630JenkinsSpringSuite970\nWestLetha,AR93555   

                 job  marital  education  default  balance  housing   loan  \
index                                             

In [33]:

from enum import Enum

class GroupByType(str, Enum):
    water = 'address_category_water'
    relief = 'address_category_relief'
    flat = 'address_category_flat'

In [43]:
GroupByType.water[:]

'address_category_water'